In [1]:
import rdflib
import pandas as pd

In [2]:
from IPython.display import HTML
df = pd.DataFrame(list(range(5)), columns=['a'])
df['a'] = df['a'].apply(lambda x: '<a href="http://example.com/{0}">link</a>'.format(x))
HTML(df.to_html(escape=False))

,a
0,link
1,link
2,link
3,link
4,link


In [6]:
import requests

dbpedia_endpoint = 'http://dbpedia.org/sparql/'

qbo_airbase_endpoint = 'http://lod.cs.aau.dk:8891/sparql'


def run_query(endpoint, query, vars):
    r = requests.get(endpoint, params={'query': query, 'format': 'application/sparql-results+json'})

    for row in r.json()['results']['bindings']:
        yield [row[v]['value'] for v in vars]


def run_all(name):
    queryDbpedia = """

SELECT DISTINCT ?city {

?city rdfs:label "%(label)s"@de . 

}""" % {'label': name}

    for city, in run_query(dbpedia_endpoint, queryDbpedia, ['city']):

        print(city)

        queryAirbase = """

PREFIX property: <http://qweb.cs.aau.dk/airbase/property/>

PREFIX owl: <http://www.w3.org/2002/07/owl#>

PREFIX schema: <http://qweb.cs.aau.dk/airbase/schema/>

SELECT ?station ?long ?lat

WHERE { 

    ?station schema:inCity ?c;

        property:longitudeDegree ?long;

        property:latitudeDegree ?lat . 

    ?c owl:sameAs <%(city)s> .

}""" % {'city': city}

        # print(queryAirbase)

        for station, long, lat in run_query(qbo_airbase_endpoint, queryAirbase, ['station', 'long', 'lat']):
            print("\t", station, long, lat)


def main():
    #run_all('Mannheim')
    run_all('Mosbach')

if __name__ == "__main__":
    main()


http://dbpedia.org/resource/Mosbach
	 http://qweb.cs.aau.dk/airbase/data/station/DEBW044_DEBW044/ 9.151447 49.363819
http://www.wikidata.org/entity/Q23008
http://www.wikidata.org/entity/Q1414547
http://www.wikidata.org/entity/Q1676840
http://www.wikidata.org/entity/Q234686


In [25]:
import requests

dbpedia_endpoint = 'http://dbpedia.org/sparql/'

openei_endpoint = 'https://openei.org/sparql/l'


def run_query(endpoint, query, vars):
    r = requests.get(endpoint, params={'query': query, 'format': 'application/sparql-results+json'})

    for row in r.json()['results']['bindings']:
        yield [row[v]['value'] for v in vars]


def run_all(name):
    queryDbpedia = """

SELECT DISTINCT ?city {

?city rdfs:label "%(label)s"@de .
?city a dbo:City . 

}""" % {'label': name}

    for city, in run_query(dbpedia_endpoint, queryDbpedia, ['city']):

        print(city)

        query_OpenEi = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
SELECT ?page ?name ?address ?place_name ?zip ?coordinates ?sector_name ?category_name
FROM <http://openei.org>
WHERE {
  ?organization swivt:page ?page.
  ?organization rdf:type category:Policy_Organizations.
  ?organization rdfs:label ?name.
 
  OPTIONAL { ?organization property:Address ?address. }.
  OPTIONAL { ?organization property:Zip ?zip. }.
  OPTIONAL { ?organization property:Coordinates ?coordinates. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
  OPTIONAL {
    ?organization property:Sector ?sector. 
    ?sector rdfs:label ?sector_name.
  }.
  OPTIONAL {
    ?organization rdf:type ?category.
    ?category rdfs:label ?category_name.
  }.
FILTER ( regex(?place_name, <%(city)s> ) )
}""" % {'city': name}

        # print(queryAirbase)

        for page, name, place_name in run_query(openei_endpoint,  query_OpenEi, ['page', 'name', 'place_name']):
            print("\t", page, name, place_name)


def main():
    #run_all('Mannheim')
    run_all('Los Angeles')

if __name__ == "__main__":
    main()


http://dbpedia.org/resource/Los_Angeles


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [113]:
DBPEDIA_GET_CITY_COUNTRY ="""
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?city ?country WHERE {
 ?city a dbo:PopulatedPlace ;
    rdfs:label "%(label)s"@en ;
    dbo:country ?country
}""" 

DBPEDIA_GET_CITY_COUNTRY_STRIPPED =""" PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT DISTINCT ?city ?country ?strippedLabel WHERE {
 ?city a dbo:PopulatedPlace  ;
    rdfs:label "%(label)s"@en ;
    dbo:country ?country.
 ?country rdfs:label ?countryLabel.
FILTER langMatches( lang(?countryLabel), "EN" ) 
BIND (STR(?countryLabel)  AS ?strippedLabel) 
}"""

OPEN_EI_QUERY_ALL_REGIONS =""" PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
PREFIX dbo: <http://dbpedia.org/ontology>
PREFIX dbr: <http://dbpedia.org/resource>

SELECT DISTINCT(?region) 
FROM <http://openei.org>
WHERE {
 ?organization property:Region ?region.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
}"""

OPEN_EI_QUERY_SPECIFIC_REGION_BY_COUNTRY_FILTER =""" 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
PREFIX dbo: <http://dbpedia.org/ontology>
PREFIX dbr: <http://dbpedia.org/resource>

SELECT DISTINCT(?region) 
FROM <http://openei.org>
WHERE {
 ?organization property:Region ?region.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
FILTER (regex(?region, "%(country)s"))
}


"""

OPEN_EI_QUERY_REGION_BY_OWL_SAME_AS_DBPEDIA = """ PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
PREFIX dbo: <http://dbpedia.org/ontology>
PREFIX dbr: <http://dbpedia.org/resource>

SELECT DISTINCT ?region
FROM <http://openei.org>
WHERE {
?region owl:sameAs <http://dbpedia.org/resource/Cologne> .
}
"""

OPEN_EI_QUERY_ALL_COUNTRYS_IN_A_COUNTRY_WHERE_REGION_EXISTS = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
PREFIX dbo: <http://dbpedia.org/ontology>
PREFIX dbr: <http://dbpedia.org/resource>
SELECT ?page ?name ?address ?place_name ?zip ?coordinates ?sector_name ?category_name
FROM <http://openei.org>
WHERE {
  ?organization swivt:page ?page.
  ?organization rdf:type category:Companies.
  ?organization rdfs:label ?name.
  ?organization property:Region <http://openei.org/resources/Germany> .
  OPTIONAL { ?organization property:Address ?address. }.
  OPTIONAL { ?organization property:Zip ?zip. }.
  OPTIONAL { ?organization property:Coordinates ?coordinates. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
  OPTIONAL {
    ?organization property:Sector ?sector. 
    ?sector rdfs:label ?sector_name.
  }.
  OPTIONAL {
    ?organization rdf:type ?category.
    ?category rdfs:label ?category_name.
  }.
}

 """

OPEN_EI_QUERY_RETRIEVE_ALL_INSTITUTES_FROM_COUNTRY_BY_PLACE_NAME = """ 

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
SELECT ?page ?name ?address ?place_name ?zip ?coordinates ?sector_name ?category_name
FROM <http://openei.org>
WHERE {
  ?organization swivt:page ?page.
  ?organization rdf:type category:Research_Institutions.
  ?organization rdfs:label ?name.
  OPTIONAL { ?organization property:Address ?address. }.
  OPTIONAL { ?organization property:Zip ?zip. }.
  OPTIONAL { ?organization property:Coordinates ?coordinates. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
  OPTIONAL {
    ?organization property:Sector ?sector. 
    ?sector rdfs:label ?sector_name.
  }.
  OPTIONAL {
    ?organization rdf:type ?category.
    ?category rdfs:label ?category_name.
  }.
FILTER (regex(?place_name, "Germany"))
}

"""

OPEN_EI_QUERY_FINANCIAL_INSTITUTIONS = """ 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
SELECT distinct ?page ?name ?address ?place_name 
FROM <http://openei.org>
WHERE {
  ?organization swivt:page ?page.
  ?organization rdf:type category:Financial_Organizations.
  ?organization rdfs:label ?name.
  OPTIONAL { ?organization property:Address ?address. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
FILTER(regex(?place_name, "%(country)s"))
}

"""

# finds 975 orgas

OPEN_EI_QUERY_ALL_ORGANIZATIONS_FROM_COUNTRY_OR_CITY = """ 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
PREFIX place: <http://openei.org/resources/Place-3A>

SELECT DISTINCT ?organization ?name ?place {
?organization rdf:type category:Organizations.
?organization rdfs:label ?name.
?organization property:Place ?place.
FILTER(regex(?place, "%(country)s"))
}

"""

OPEN_EI_QUERY_POLICY_ORGANIZATIONS_PER_COUNTRY = """ 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
SELECT DISTINCT ?page ?name ?address ?place_name 
FROM <http://openei.org>
WHERE {
  ?organization swivt:page ?page.
  ?organization rdf:type category:Policy_Organizations.
  ?organization rdfs:label ?name.
  OPTIONAL { ?organization property:Address ?address. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
  OPTIONAL {
    ?organization property:Sector ?sector. 
    ?sector rdfs:label ?sector_name.
  }.
FILTER(regex(?place_name, "%(country)s"))
}
"""


OPEN_EI_QUERIES_FIND_TOOLS = """ 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
SELECT DISTINCT(?organization) ?name ?address ?place_name 
FROM <http://openei.org>
WHERE {
  ?organization rdf:type category:Tools.
  ?organization rdfs:label ?name.
  OPTIONAL { ?organization property:Address ?address. }.
  OPTIONAL { ?organization property:Zip ?zip. }.
  OPTIONAL { ?organization property:Coordinates ?coordinates. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
  OPTIONAL {
    ?organization property:Sector ?sector. 
    ?sector rdfs:label ?sector_name.
  }.
  OPTIONAL {
    ?organization rdf:type ?category.
    ?category rdfs:label ?category_name.
  }.
FILTER(regex(?place_name, "%(country)s"))
}

"""

OPEN_EI_CLEAN_ENERGY_COMPANIES =""" 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX swivt: <http://semantic-mediawiki.org/swivt/1.0#>
PREFIX resource: <http://openei.org/resources/>
PREFIX category: <http://openei.org/resources/Category-3A>
PREFIX property: <http://openei.org/resources/Property-3A>
SELECT DISTINCT(?organization) ?name ?address ?place_name 
FROM <http://openei.org>
WHERE {
  ?organization rdf:type category:Companies.
  ?organization rdfs:label ?name.
  OPTIONAL { ?organization property:Address ?address. }.
  OPTIONAL { ?organization property:Zip ?zip. }.
  OPTIONAL { ?organization property:Coordinates ?coordinates. }.
  OPTIONAL {
    ?organization property:Place ?place.
    ?place rdfs:label ?place_name.
  }.
  OPTIONAL {
    ?organization property:Sector ?sector. 
    ?sector rdfs:label ?sector_name.
  }.
  OPTIONAL {
    ?organization rdf:type ?category.
    ?category rdfs:label ?category_name.
  }.
FILTER(regex(?place_name, "%(country)s"))
}

"""

OPEN_EI_CLEAN_ENERGY_COMPANIES_PARAMS = ['organization', 'name','place_name' ]
OPEN_EI_ORGANIZATIONS_PARAMS = ['organization', 'name','place' ]
OPEN_EI_QUERY_POLICY_ORGANIZATIONS_PER_COUNTRY_PARAMS = ['page', 'name', 'address', 'place_name']
#OPEN_EI_QUERY_POLICY_ORGANIZATIONS_PER_COUNTRY_DICT = {'page':a, 'name':b, 'address':c, 'place_name':d, 'zip':e, 'coordinates':f, 'sector_name':g, 'category_name':h}
OPEN_EI_ORGANIZATION_PARAMS = ['page', 'name', 'place_name', 'address']
OPEN_EI_FINANCIAL_PARAMS = ['page' ,'name', 'address', 'place_name' ]

### Run Query General

In [35]:
import requests
import re
def run_query(endpoint, query, vars):
    r = requests.get(endpoint, params={'query': query, 'format': 'application/sparql-results+json', 'charset':'utf-8'})
    for row in r.json()['results']['bindings']:
        #[print(row[v]['value']) for v in vars if v in row]      
        yield [row[v]['value'] for v in vars if v in row]     

#### Clean Energy Company by Country

In [126]:
dbpedia_endpoint = 'http://dbpedia.org/sparql/'
openei_endpoint = 'https://openei.org/sparql/l'


def run_clean_energy_company_by_country(name):
    results = pd.DataFrame()
    queryDbpedia = DBPEDIA_GET_CITY_COUNTRY_STRIPPED % {'label': name}
    
    for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
        query_OpenEi = OPEN_EI_CLEAN_ENERGY_COMPANIES % {'country': name}
        
        for a, b, c in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_CLEAN_ENERGY_COMPANIES_PARAMS):
            results = results.append(pd.Series({'organization':a,'name':b,'place_name':c}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    
    print("Found {} clean energy organizations in {}".format(len(results), name))
    results = pd.DataFrame()    
   
    query_OpenEi = OPEN_EI_CLEAN_ENERGY_COMPANIES % {'country': strippedLabel}
        
    for a, b, c in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_CLEAN_ENERGY_COMPANIES_PARAMS):
        results = results.append(pd.Series({'organization':a,'name':b,'place_name':c}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    print("In total found {} clean energery organizations in {}".format(len(results),strippedLabel))
    return results


pandas_ = run_clean_energy_company_by_country('Arnstadt')
pandas_['organization'] = pandas_['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x))#pandas_['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)) #get rid of lod page  lod/page
print(pandas_)

Found 3 clean energy organizations in Arnstadt
In total found 239 clean energery organizations in Germany
                                                  name  \
0                                  8KU Renewables GmbH   
1                             AEROGIE Verwaltungs GmbH   
2    Antec Solar Energy AG formerly Oekologik Ecove...   
3                                            Atmosfair   
4                                          BGI Ecotech   
5                                             Altus AG   
6       ASEM Green Independent Power Producers Network   
7                                  BASF Linde Group JV   
8                                                  ADT   
9                Aachen University of Applied Sciences   
10                                 ASi Industries GmbH   
11                                       AgriKomp GmbH   
12                  Allianz Climate Solutions ACS GmbH   
13                                          Amann GmbH   
14                      

### Tools y Coutry

In [127]:
dbpedia_endpoint = 'http://dbpedia.org/sparql/'
openei_endpoint = 'https://openei.org/sparql/l'


def run_tools_by_country(name):
    results = pd.DataFrame()
    queryDbpedia = DBPEDIA_GET_CITY_COUNTRY_STRIPPED % {'label': name}
    for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
        query_OpenEi = OPEN_EI_QUERIES_FIND_TOOLS % {'country': name}
        
        for a, b, c in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_CLEAN_ENERGY_COMPANIES_PARAMS):
            results = results.append(pd.Series({'organization':a,'name':b,'place_name':c}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    
    print("Found {} tools in {}".format(len(results), name))
    results = pd.DataFrame()       
   
    query_OpenEi = OPEN_EI_QUERIES_FIND_TOOLS % {'country': strippedLabel}

    for a, b, c in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_CLEAN_ENERGY_COMPANIES_PARAMS):
        results = results.append(pd.Series({'organization':a,'name':b,'place_name':c}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    print("In total found {} tools in {}".format(len(results),strippedLabel))
    return results


pandas_ =  run_tools_by_country('Mosbach')
pandas_['organization'] = pandas_['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x))#pandas_['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)) #get rid of lod page  lod/page


print(pandas_)

Found 0 tools in Mosbach
In total found 15 tools in Germany
                                                 name  \
0                ESMAP-Energy Efficiency Case Studies   
1                          Carbon and Energy Reporter   
2                      Continuous Diagnostics Advisor   
3                          Energy Performance Monitor   
4   Carbon Dioxide Information Analysis Center (CD...   
5                          Ecofys-Country Fact Sheets   
6   Energy Technology Systems Analysis Program (MA...   
7   Greenhouse Gas Emission Trends and Projections...   
8   Grid-Connected Renewable Energy Systems Case S...   
9                   Low Carbon Society (LCS) Database   
10                     Policies for Low Carbon Growth   
11  Renewable Energy Projections as Published in t...   
12  IEA Wind Task 26 - Multi-national Case Study o...   
13                           OECD Input-Output Tables   
14                          UNFCCC-Global Map-Annex 1   

                           

### Policy Organizations


In [61]:
dbpedia_endpoint = 'http://dbpedia.org/sparql/'
openei_endpoint = 'https://openei.org/sparql/l'


def run_policy_by_country(name):
    results = pd.DataFrame()
    queryDbpedia = DBPEDIA_GET_CITY_COUNTRY_STRIPPED % {'label': name}
    
    for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
        query_OpenEi = OPEN_EI_QUERY_POLICY_ORGANIZATIONS_PER_COUNTRY % {'country': name}
        
        for a, b, c, d in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_QUERY_POLICY_ORGANIZATIONS_PER_COUNTRY_PARAMS):
            results = results.append(pd.Series({'page':a, 'name':b, 'address':c, 'place_name':d}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    print("Found {} policy organizations in {}".format(len(results), name))
    results = pd.DataFrame()
    
    for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
        query_OpenEi = OPEN_EI_QUERY_POLICY_ORGANIZATIONS_PER_COUNTRY % {'country': strippedLabel}
        

        for a, b, c, d in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_QUERY_POLICY_ORGANIZATIONS_PER_COUNTRY_PARAMS):
            results = results.append(pd.Series({'page':a, 'name':b, 'address':c, 'place_name':d}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
  
    print("In total found {} policy organization in {}".format(len(results),strippedLabel))
    return results



pandas_ =  run_policy_by_country('Rome')

print(pandas_)

Found 1 policy organizations in Rome
In total found 2 policy organization in Italy
                                             address  \
0  International Fund for Agricultural Developmen...   
1           Island of San Giorgio Maggiore\nI-30124,   

                                              name  \
0  International Fund for Agricultural Development   
1      International Center for Climate Governance   

                                                page  \
0  https://openei.org/wiki/International_Fund_for...   
1  https://openei.org/wiki/International_Center_f...   

                        place_name  
0    Rome, Italy: Energy Resources  
1  Venice, Italy: Energy Resources  


### Orgaizatios i City or Coutry

In [128]:
dbpedia_endpoint = 'http://dbpedia.org/sparql/'
openei_endpoint = 'https://openei.org/sparql/l'


def run_organization_by_country_or_city(name):
    results = pd.DataFrame()
    queryDbpedia = DBPEDIA_GET_CITY_COUNTRY_STRIPPED % {'label': name}
    
    for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
        query_OpenEi = OPEN_EI_QUERY_ALL_ORGANIZATIONS_FROM_COUNTRY_OR_CITY % {'country': name}
        
        for a, b, c in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_ORGANIZATIONS_PARAMS):
            results = results.append(pd.Series({'organization':a,'name':b,'place':c}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    
    print("Found {} organizations in {}".format(len(results), name))
    if len(results) < 2:
        results = pd.DataFrame()    
        
        query_OpenEi = OPEN_EI_QUERY_ALL_ORGANIZATIONS_FROM_COUNTRY_OR_CITY  % {'country': strippedLabel}

        for a, b, c in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_ORGANIZATIONS_PARAMS):
            results = results.append(pd.Series({'organization':a,'name':b,'place':c}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    print("In total found {} organizations in {}".format(len(results),strippedLabel))
    return results


pandas_ = run_organization_by_country_or_city('Cologne')
pandas_['organization'] = pandas_['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x))#pandas_['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)) #get rid of lod page  lod/page
pandas_['place'] = pandas_['place'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x))#pandas_['organization'].apply(lambda x: re.sub('http://openei.org/resources', 'https://openei.org/wiki', x)) #get rid of lod page  lod/page

print(pandas_)

Found 4 organizations in Cologne
In total found 4 organizations in Germany
                                              name  \
0  Concentrating Solar Power Services CSP Services   
1                                        Solare AG   
2                          Ubbink Solar Modules BV   
3                    German Aerospace Center (DLR)   

                                        organization  \
0  https://openei.org/wiki/Concentrating_Solar_Po...   
1                  https://openei.org/wiki/Solare_AG   
2    https://openei.org/wiki/Ubbink_Solar_Modules_BV   
3  https://openei.org/wiki/German_Aerospace_Cente...   

                                               place  
0           https://openei.org/wiki/Cologne,_Germany  
1  https://openei.org/wiki/Cologne,_North_Rhine-2...  
2  https://openei.org/wiki/Cologne,_North_Rhine-2...  
3           https://openei.org/wiki/Cologne,_Germany  


In [ ]:
HOH

In [10]:
dbpedia_endpoint = 'http://dbpedia.org/sparql/'
openei_endpoint = 'https://openei.org/sparql/l'


def run_financial_by_country_or_city(name):
    results = pd.DataFrame()
    queryDbpedia = DBPEDIA_GET_CITY_COUNTRY_STRIPPED % {'label': name}
    
    for city,country, strippedLabel in run_query(dbpedia_endpoint, queryDbpedia, ['city', 'country', 'strippedLabel']):
        query_OpenEi = OPEN_EI_QUERY_FINANCIAL_INSTITUTIONS % {'country': name}
        
        for a, b, c, d in run_query(openei_endpoint,  query_OpenEi, OPEN_EI_FINANCIAL_PARAMS):
            results = results.append(pd.Series({'page':a,'name':b,'address':c, 'place_name':d}), ignore_index=True) #results.append(pd.Series(data=[a,b,c]), ignore_index=True)
    
    print("Found {} FINANCIAL_INSTITUTIONS in {}".format(len(results), name))
    return results


pandas_ = run_financial_by_country_or_city('Munich')
print(pandas_)

Found 1 FINANCIAL_INSTITUTIONS in Munich
             address               name  \
0  Schumannstrasse 4  Perseus (Germany)   

                                        page  \
0  https://openei.org/wiki/Perseus_(Germany)   

                          place_name  
0  Munich, Germany: Energy Resources  
